# Performance: Runtimes

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

root_dir = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
stats_dir = os.path.join(root_dir, "assets/stats")
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)
    
from topologiq.scripts.pathfinder import test_pthfinder

## 1. Introduction

This notebook aims to offer a means to test Topologiq's runtimes. It starts with checks for the pathfinder's single edge construction, then moves into full circuit generation.

The expectations are:
- Runtimes should increase linearly against the total number of sites visited in any given iteration of the pathfinder algorithm,
- Runtimes should increase exponentially against the radius of the space searched in any given iteration of the pathfinder algorithm. 

## 2. Single edges

Let's start with some tests designed to test the pathfinder's core function by asking it to make paths between all kinds of cubes and all kinds of cubes, with cubes placed at different Manhattan distances from one another.

Internally, the function below calls the pathfinder with a unique run identifier via the optional parameter `log_stats_id`. The identifier has a "*" at the end to prompt stats are saved to a file specifically for tests as opposed to the file used to tracking stats for generation of paths as part of full circuit operations.

*Please note the block below calls the algorithm over 300 times. It will take a while! A few minutes, perhaps.*

In [ ]:
max_test_step = 9
num_repetitions = 10
min_succ_rate = 100
test_pthfinder(
    stats_dir,
    min_succ_rate,
    max_test_step=max_test_step,
    num_repetitions=num_repetitions,
)

Before visualising the data, we will filter it to control for the following factors:
- Ignore failures (the outer graph manager algorithm has ways to deal with individual failures by the pathfinder, but the code above tests only the pathfinder so runtimes of failed runs would bias statistics),
- Ignore boundary ("O") nodes (the algorithm deals with "O" nodes with something of a a shortcut, so their runtimes would bias runtimes statistics),
- Ignore runtimes equal to 0 seconds (since the algorithm is fast, the computer's clock will not always clock it).

In [ ]:
# Load data
df_iter = pd.read_csv(os.path.join(stats_dir, "pathfinder_iterations_tests.csv"), delimiter=";")
df_iter_filt = df_iter[(df_iter["iter_success"] == True) & (df_iter["tgt_zx_type"] != "O")]

Now we can proceed to visualise the data and, in particular, the relation between:
- Visitation attempts and runtimes: the algorithm only places a block if the coordinate/block combination clears a number of checks, so the purest reflection of the number of times the algorithm runs most of its operations is the number of times it "visits" a "site" and tries to place a block in it irrespective of whether the block is ultimately placed in the site or not.
- Length of longest path in run and runtimes: the closest to a "radius" for the pathfinder algorithm is not the pure Manhattan distance between a source and a target but the length of the longest path generated in any given run of the algorithm, as this path subsumes the 4th (kind) dimension visited by the algorithm in the form of extra length relatively to a direct path between source and target.

In [ ]:
# SERIES, LABELS, & TITLES
series = ["num_visitation_attempts", "len_longest_path"]
x_labels = ["No. of visitation attempts", "Le. of longest path"]
titles = ["Duration v. Visitation attempts", "Duration v. Len. of longest path"]

# MATPLOTLIB VISUALISATION
fig, axes = plt.subplots(nrows=1, ncols=len(series), figsize=(7 * len(series), 5))
fig.suptitle("Pathfinder runtimes (single segments)", fontsize=18)
for i, series_name in enumerate(series):

    axes[i].scatter(
        df_iter_filt[series_name],
        df_iter_filt["iter_duration"],
        marker="x",
        color="b",
        label="Individual durations",
    )

    series_ave_dur = df_iter_filt.groupby(series_name)["iter_duration"].mean()
    axes[i].plot(
        series_ave_dur.index,
        series_ave_dur.values,
        marker="o",
        linestyle="--",
        color="r",
        label="Average duration",
    )

    min_x = int(df_iter_filt[series_name].min())
    max_x = int(df_iter_filt[series_name].max())
    axes[i].set_title(titles[i], fontsize=14)
    axes[i].set_xlabel(x_labels[i], fontsize=10)
    axes[i].set_ylabel("Average duration (seconds)", fontsize=10)
    axes[i].set_xticks(ticks=range(min_x, max_x + 1, max(1, int((max_x - min_x) / 10))))
    axes[i].grid(True, which="both", linestyle="--", linewidth=0.5)
    axes[i].legend()

# GRAPH VISUAL ADJUSTMENTS
plt.tight_layout()

# SAVE PLOT TO FILE (Uncomment to save plot to file)
# plt.savefig("pathfinder_duration_edges_from_full_circuit_runs")

# SHOW PLOT
plt.show()

In all runs of this notebook we have done by ourselves, the charts above match expectations: 
- There is a clear linear relation between the number of sites visited and the runtimes, 
- There is a seemingly exponential relation between the the lenght of the longest path (the closest thing available to a "radius" for this algorithm, as it considers types by means of extra path lenght relative to a pure Manhattan distance) and runtimes.

It can also be observed from the figures that iteration times tend to stick together, which sugggests runtimes will be homogeneous given homogeneous conditions.

## 2. Assembling real circuits

This section relies on the existence of data about regular runs of the full algorithmic cycle. Check the files `./assets/stats/pathfinder_iterations.csv` and `./assets/stats/bfs_manager.csv` to ensure you have data to work with.

Topologiq *can* regularly log stats about the following aspects of any normal runs:
- Metrics about the performance of the pathfinder algorithm, for each run of the pathfinder, 
- Metrics about the performance of the full algorithmic cycle, including runtimes for the full process and data about the original graphs and any outputs.

The main runner function *can* receive an optional (boolean) parameter called `log_stats`. If `log_stats` is set to `True`, the graph manager will turn logging on pass a unique identifier around to ensure all logs can be connected.

It is good practice to log stats from a few runs here and there, which should, overtime, produce a good sample reflecting varied usage conditions and circuits.

***That said,*** if you have never run the algorithm with stats logs enabled and still want to run this notebook, you can append `--log_stats` to any of the examples in the [README](../../README.md), as well as `--repeat:50` so the example is run 50 times automatically. For a good sample, you will need to do this for several examples. Refer to the [README](../../README.md) for command examples.

### 2.1. Pathfinder

Let's start by repeating the previous analysis with data from real runs of the entire algorithmic process. 

The first step is, of course, loading the data about iterations by the core pathfinder algorithm.

In [ ]:
df_iter_full_circuits = pd.read_csv(os.path.join(stats_dir, "pathfinder_iterations.csv"), delimiter=";")
df_filt_full_circuits = df_iter_full_circuits[
    (df_iter_full_circuits["iter_success"] == True)
    & (df_iter_full_circuits["tgt_zx_type"] != "O")
]

It is then possible to visualise this data similarly to how it was done with the data about single segments.

In [ ]:
# SERIES, LABELS, & TITLES
series = ["num_visitation_attempts", "len_longest_path"]
x_labels = ["No. of visitation attempts", "Le. of longest path"]
titles = ["Duration v. Visitation attempts", "Duration. v. Len. of longest path"]

# MATPLOTLIB VISUALISATION
fig, axes = plt.subplots(nrows=1, ncols=len(series), figsize=(7 * len(series), 5))
fig.suptitle("Pathfinder runtimes (segments from full circuits)", fontsize=18)
for i, series_name in enumerate(series):

    axes[i].scatter(
        df_filt_full_circuits[series_name],
        df_filt_full_circuits["iter_duration"],
        marker="x",
        color="b",
        label="Individual durations",
    )

    series_ave_dur = df_filt_full_circuits.groupby(series_name)["iter_duration"].mean()
    axes[i].plot(
        series_ave_dur.index,
        series_ave_dur.values,
        marker="o",
        linestyle="--",
        color="r",
        label="Average duration",
    )

    min_x = int(df_filt_full_circuits[series_name].min())
    max_x = int(df_filt_full_circuits[series_name].max())
    axes[i].set_title(titles[i], fontsize=14)
    axes[i].set_xlabel(x_labels[i], fontsize=10)
    axes[i].set_ylabel("Duration (seconds)", fontsize=10)
    axes[i].set_xticks(ticks=range(min_x, max_x + 1, max(1, int((max_x - min_x) / 10))))
    axes[i].grid(True, which="both", linestyle="--", linewidth=0.5)
    axes[i].legend()

# GRAPH VISUAL ADJUSTMENTS
plt.tight_layout()

# SAVE PLOT TO FILE (Uncomment to save plot to file)
# plt.savefig("pathfinder_duration_edges_from_full_circuit_runs")

# SHOW PLOT
plt.show()

In all runs of this notebook we have done by ourselves, the charts above show more variability due to coming from runs of different circuits, but the fundamental patterns match expectations: 
- The relation between the number of sites visited and the runtimes remains linear, 
- The relation between the the lenght of the longest path and runtimes seems exponential.

### 2.2 Full cycle

When stats logging is enabled, ***topologiq*** will also save data about the full algorithmic cycle. Let's check runtimes for the entire process! 

Firstly, let's load the data.

We will:
- Filter it to, for now, to include only successful runs,
- Add a column for the node-to-edge ratio of the incoming ZX circuit (numbers considerably below 1 indicate a dense graph with more edges than nodes),
- Filter outliers by groups, to avoid biasing results with excessive delays in runtimes that do not reflect usual performance (typically caused by some sort of computer or user glitch such as, for instance, closing the computer while the process is running and having the process finish hours later).

In [ ]:
df_cycle = pd.read_csv(os.path.join(stats_dir, "graph_manager_cycle.csv"), delimiter=";")
df_cycle_filt = df_cycle[df_cycle["run_success"] == True].copy()
df_cycle_filt["node_edge_ratio"] = np.where(df_cycle_filt["num_input_nodes_processed"] != 0, df_cycle_filt["num_input_nodes_processed"] / df_cycle_filt["num_input_edges_processed"], 0)

In this case, we will visualise runtimes by several factors:
- By the number of nodes in the original graph, which roughly shows the size of the original circuit
- By the node-to-edge ratio in the original graph, which communicates the density of the original graph (ratios below 1: graph had more edges than nodes)
- By the number of cubes in the final ouput, a proxy of how many operations the algorithm had to take to assemble the circuit.
- By the name of the circuit, which aggregates performance over many runs of the same circuit.

Please note that the numbers aggregate performance for often-very-different versions of the same circuit. The best performing runs for any given circuit can be as much as twice as fast as the worst performing runs. This is non-ideal, but it reflects the current state of affairs: ***topologiq*** does not currenlty have a mechanism to ensure the best performing version of a circuit is always returns.

In [ ]:
# FUNCTION TO REMOVE OUTLIERS BY GROUPS
def remove_outliers(df, series_name):
    q1 = df.groupby(series_name)["duration_total"].transform("quantile", 0.05)
    q3 = df.groupby(series_name)["duration_total"].transform("quantile", 0.95)
    iqr = q3 - q1

    low = q1 - 1.5 * iqr
    up = q3 + 1.5 * iqr

    df_clean = df[
        (df["duration_total"] >= low)
        & (df["duration_total"] <= up)
    ]
    
    return df_clean


# SERIES, LABELS, & TITLES
series = [["num_input_nodes_processed", "node_edge_ratio"], ["num_blocks_output", "circuit_name"]]
x_labels = [
    ["No. nodes/spiders (input)", "Node/edge ratio (input)"],
    ["Num. blocks (output)", "Name of circuit"],
]
titles = [
    [
        "Duration v. Num nodes in input graph",
        "Duration v. Nodes-to-edge ration of input graph",
    ],
    ["Duration v. Num blocks in output", "Duration v. Name of circuit"],
]

# MATPLOTLIB VISUALISATION
fig, axes = plt.subplots(nrows=2, ncols=len(series), figsize=(7 * len(series), 6))
fig.suptitle("Pathfinder runtimes (segments from full circuits)", fontsize=18)

for i, row in enumerate(series):
    
    for j, series_name in enumerate(row):
        
        df_cycle_filt_clean = remove_outliers(df_cycle_filt, series_name)
        
        if i != 1 or j != 1: 
            axes[i][j].scatter(
                df_cycle_filt_clean[series_name],
                df_cycle_filt_clean["duration_total"],
                marker="x",
                color="b",
                label="Individual durations",
            )

            series_ave_dur = df_cycle_filt_clean.groupby(series_name)[
                "duration_total"
            ].mean()
            axes[i][j].plot(
                series_ave_dur.index,
                series_ave_dur.values,
                marker="o",
                linestyle="--",
                color="r",
                label="Average duration",
            )
            
            axes[i][j].set_title(titles[i][j], fontsize=10)
            axes[i][j].set_xlabel(x_labels[i][j])
            axes[i][j].set_ylabel("Duration (seconds)") 

        else:
            axes[i][j].scatter(
                df_cycle_filt_clean["duration_total"],
                df_cycle_filt_clean[series_name],
                marker="x",
                color="b",
                label="Individual durations",
            )
             
            axes[i][j].set_title(titles[i][j], fontsize=14)
            axes[i][j].set_xlabel("Duration (seconds)", fontsize=10)
            axes[i][j].set_ylabel(x_labels[i][j], fontsize=10) 
        
       
        axes[i][j].grid(True, which="both", linestyle="--", linewidth=0.5)

# GRAPH VISUAL ADJUSTMENTS
plt.tight_layout()

# SAVE PLOT TO FILE (Uncomment to save plot to file)
# plt.savefig("pathfinder_duration_edges_from_full_circuit_runs")

# SHOW PLOT
plt.show()

At the moment, the only thing that can be inferred from the above with some degree of certainty is that there is significantly variability of runtimes across circuits of different kinds to enable hard conclusions from any of the above. 

There is a slight upwards trend in the relation between number of nodes in input and output and the runtimes. This makes sense. The more nodes and edges in the incoming ZX graph, the more nodes and edges there are for processing. However, some of the runs of circuits with 16 nodes stop the clock at around similar times as circuits with 10 nodes. This might suggest runtimes are informed partially by the size of the incoming and by extension outgoing graph, but that other factors also matter. 

Additional research is needed to determine the additional factors that influence times.

Preliminarily, it might be worth to look into the reasons for the variability related to the node-to-edge ratios and the different circuits (especially the "simple mess", which has quite a broad range of possible runtime values).